In [3]:
from keras.models  import load_model
import keras.backend as K

import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer

import pandas as pd
import numpy as np

from support import norm
aspects = [str(i) for i in range(4)]

Using TensorFlow backend.


In [4]:
vec = cPickle.load(open('../../beer_data/beer_vec_ds_2000.p', 'rb'))

In [5]:
X = vec.X

In [6]:
model = load_model('../store/weights/beer/ThuNov220:01:282017/loss.h5', 
                   custom_objects={'contrastive_loss' : lambda a,b:b})

In [7]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
O (InputLayer)                   (None, 305)           0                                            
____________________________________________________________________________________________________
S0 (InputLayer)                  (None, 305)           0                                            
____________________________________________________________________________________________________
D0 (InputLayer)                  (None, 305)           0                                            
____________________________________________________________________________________________________
S1 (InputLayer)                  (None, 305)           0                                            
___________________________________________________________________________________________

In [8]:
model_c1 = {}
for aspect in aspects :
    model_c1[aspect] = K.function(model.get_layer('pool_'+aspect).inputs + [K.learning_phase()], 
                          model.get_layer('pool_'+aspect).outputs)

In [12]:
samples = np.random.choice(len(X), 500, replace=False)
X_s = X[samples]
embeds = {}
for aspect in aspects :
    embeds[aspect] = model_c1[aspect]([X_s, 0])[0]

In [13]:
ds = pd.read_csv('../../beer_data/beer_ds.csv')
from ast import literal_eval as make_tuple
ds['bits'] = ds['bits'].map(lambda s : make_tuple(s))
val_idxs = samples
H = {}
aspect_columns = sorted(['review/appearance', 'review/taste', 'review/aroma', 'review/palate'])

for i in range(4) :
    H[str(i)] = np.zeros((len(val_idxs), len(val_idxs)))
    a0 = set(ds[ds[aspect_columns[i]] == 0].index) & set(val_idxs)
    a1 = set(ds[ds[aspect_columns[i]] == 1].index) & set(val_idxs)
    a0 = map(lambda s : list(val_idxs).index(s), a0)
    a1 = map(lambda s : list(val_idxs).index(s), a1)
    for j in a0 :
        H[str(i)][j, a0] = 1
    for j in a1 :
        H[str(i)][j, a1] = 1

    H[str(i)][np.arange(len(val_idxs)), np.arange(len(val_idxs))] = 0

In [14]:
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score
for aspect in aspects :
    result = embeds[aspect]
    #result = np.concatenate(embeds[aspect], axis=0)
    result = normalize(result, 'l2')
    scores = np.dot(result, result.T)
    scores[np.arange(len(samples)), np.arange(len(samples))] = -1000
    aucs = [0] * len(samples)
    for i in range(len(samples)) :
        aucs[i] = roc_auc_score(H[aspect][i], scores[i])
    print aspect, np.mean(aucs)

0 0.556996197958
1 0.524092064041
2 0.514808763711
3 0.489425963469


In [ ]:
input = {}
input['O'] = X_s[0:1, :]
vals = []
for aspect in aspects :
    for i in range(1, len(X_s)) :
        input['S'+aspect] = X_s[i:(i+1), :]
        input['D'+aspect] = X_s[i:(i+1), :]
        vals.append(model.predict[input])
    
ds.iloc[[0,5,7]]

In [22]:
ds.groupby(['beer/style', 'bits']).size()

beer/style                 bits        
 Altbier                   (0, 0, 0, 0)      26
                           (0, 0, 1, 0)       1
                           (0, 1, 1, 1)       1
                           (1, 0, 0, 0)      15
                           (1, 0, 1, 0)       2
                           (1, 0, 1, 1)       5
                           (1, 1, 0, 0)      10
                           (1, 1, 0, 1)       1
                           (1, 1, 1, 0)       2
                           (1, 1, 1, 1)     235
 American Adjunct Lager    (0, 0, 0, 0)    5650
                           (0, 0, 0, 1)       2
                           (0, 0, 1, 0)      23
                           (0, 0, 1, 1)       3
                           (0, 1, 0, 0)       4
                           (0, 1, 0, 1)       1
                           (0, 1, 1, 0)       1
                           (1, 0, 0, 0)     171
                           (1, 0, 1, 0)      12
                           (1, 0, 1, 1)       5
